#  Лабораторная №2. Векторное представление текстов

### Скачайте данные отзывов о банковских услугах, используемые в лабораторной работе №1. Можно взять сэмп данных из ЛР № 1.

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/financial_review.csv')
data_bank = data.loc[data['bank_name'] == 'тинькофф банк']

data_bank.head()

,bank_href,login,review,bank_name,site,date,message_href,score,status,user_href
2487,/tinkoff-kreditnye-sistemy.html,NaN,"С финансами было все не плохо,была работа у му...",тинькофф банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN
2488,/tinkoff-kreditnye-sistemy.html,NaN,Вчера закрывал задолженность по карте Тинькофф...,тинькофф банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN
2489,/tinkoff-kreditnye-sistemy.html,NaN,"Вот так! Я клиент банка уже почти год, имею и ...",тинькофф банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN
2490,/tinkoff-kreditnye-sistemy.html,NaN,"Хороший надежный Банк, клиентом которого я явл...",тинькофф банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN
2491,/tinkoff-kreditnye-sistemy.html,NaN,19.11.2016 у меня были украдены деньги с карты...,тинькофф банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN


### Постройте векторные представления отзывов с использованием подходов BoW и TF-IDF c разными параметрами (использование n-gramm, различные пороги по отсечению частотных слов), а также с использованием предобучененных векторов word2vec.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()

vectorizer.fit(data_bank['review'])
print(vectorizer.vocabulary_)

vector = vectorizer.transform(data_bank['review'])
print(vector.toarray())
print(vector.shape)

{'финансами': 22756, 'было': 2178, 'все': 3199, 'не': 10503, 'плохо': 14131, 'была': 2174, 'работа': 17744, 'мужа': 9677, 'постоянный': 15669, 'доход': 5262, 'но': 11202, 'большой': 1976, 'муж': 9676, 'работал': 17752, 'по': 14164, 'дней': 4753, 'неделю': 10621, 'приносил': 16676, 'от': 12606, '600': 495, '3000': 330, 'рублей': 18803, 'день': 4556, 'когда': 7930, 'как': 7548, 'накопить': 10016, 'получалось': 15094, 'решили': 18666, 'попробовать': 15418, 'одобрили': 11991, '12': 71, 'тысяч': 21752, 'тогда': 21467, 'беременна': 1698, 'надо': 9862, 'кровать': 8550, 'ребёнку': 18452, 'коляску': 8013, 'потом': 15756, 'положение': 15046, 'финансовое': 22766, 'изменилось': 6969, 'до': 4765, 'ужаса': 22074, 'без': 1611, 'работы': 17784, 'помогала': 15233, 'мама': 9121, 'могла': 9516, 'голоду': 4017, 'умерли': 22254, 'возможности': 2926, 'платить': 14073, 'сами': 18921, 'может': 9535, 'виноваты': 2647, 'ещё': 5508, 'этом': 23656, 'первый': 13512, 'раз': 17835, 'позвонили': 14824, 'из': 6884, 'б

Теперь используем этот же способ векторизации, но после удаления стоп слов и лемматизации текста.

Лемматизация будет происходить так же, как и в первой лабораторной работе, с помощью pymorphy2.

In [ ]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 55.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=6796f39a38238ef46c28707a4648cee8f7a9a9ad84cf75e820eece6c4e1decff
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import pymorphy2

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords

import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
morph = pymorphy2.MorphAnalyzer()

nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def lemmatize_tokenize_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.isalnum() and token.lower() not in stop_words]
    lemmatized_tokens = [morph.parse(token)[0].normal_form for token in tokens]

    return lemmatized_tokens

In [ ]:
data_bank_lemma = data_bank.copy()

data_bank_lemma['review'] = data_bank_lemma['review'].apply(
                                            lambda x: lemmatize_tokenize_text(x))
data_bank_lemma_word2vec = data_bank_lemma.copy()
# data_bank['review'] = data_bank['review'].apply(lambda x: ' '.join(x))

# data_bank_lemma['review']

In [ ]:
data_bank_lemma['review'] = data_bank_lemma['review'].apply(lambda x: ' '.join(x))
data_bank_lemma['review']

2487     финансы плохобыть работа муж постоянный доход ...
2488     вчера закрывать задолженность карта тинькофф о...
2489     клиент банк год иметь дебетовый кредитный карт...
2490     хороший надёжный банк клиент который являться ...
2491     19112016 украсть деньга карта писать коротко с...
                               ...                        
19920    довольный клиент банкаоколо 5ти год являться к...
19921    кредитный карта tinkoff bank platinumпользоват...
19922    итак договор расторгнуть 14 май 2015 лишь 2 ме...
19923    хамский отношение сотрудник доведение пожилое ...
19924    здравствуйтеуважать сотрудник банкакто смочь п...
Name: review, Length: 1476, dtype: object

#### BoW

In [ ]:
vectorizer = CountVectorizer()

vectorizer.fit(data_bank_lemma['review'])
print(vectorizer.vocabulary_)

vector = vectorizer.transform(data_bank_lemma['review'])
print(vector.toarray())

# итого есть 1476 отзывов и 14890 терминов
print(vector.shape)

{'финансы': 14237, 'плохобыть': 9493, 'работа': 11261, 'муж': 7019, 'постоянный': 10251, 'доход': 4262, 'большой': 2237, 'работать': 11269, '36': 782, 'день': 3757, 'неделя': 7522, 'приносить': 10722, '6003000': 970, 'трубль': 13649, 'денькогда': 3782, 'какнакопить': 5550, 'получаться': 9966, 'решить': 11759, 'попробовать': 10138, 'одобрить': 8345, '12': 189, 'тысяча': 13703, 'беременный': 2069, 'кровать': 6260, 'ребёнок': 11651, 'коляска': 5937, 'положение': 9942, 'финансовый': 14236, 'измениться': 5199, 'ужас': 13869, 'работыпомогать': 11292, 'мама': 6628, 'мочь': 6997, 'голод': 3445, 'умереть': 13941, 'возможность': 2798, 'платить': 9454, 'сам': 11953, 'виноватый': 2637, 'ещё': 4424, 'первый': 9152, 'позвонить': 9824, 'банк': 1755, 'лежать': 6408, 'отделение': 8739, 'роддомародить': 11789, '20': 442, 'минута': 6843, 'назад': 7155, 'звонок': 5006, 'угроза': 13801, 'адрес': 1528, 'мой': 6934, 'шокедовести': 14668, 'слеза': 12349, 'вроде': 3001, 'договориться': 4042, 'минимальный': 682

In [ ]:
# создадим словарь, учитывающий только первые 7000 уникальных терминов
# посмотрим, как изменятся частоты слов при добавлении этого параметра
vectorizer = CountVectorizer(max_features=7000)

vectorizer.fit(data_bank_lemma['review'])
print(vectorizer.vocabulary_)

vector = vectorizer.transform(data_bank_lemma['review'])
print(vector.toarray())

# проверим, выводит ли нужное количество терминов, или нет
print(vector.shape)

{'финансы': 6737, 'плохобыть': 4026, 'работа': 5177, 'муж': 2990, 'постоянный': 4475, 'доход': 1853, 'большой': 1029, 'работать': 5178, '36': 247, 'день': 1669, 'неделя': 3214, 'приносить': 4806, '6003000': 317, 'трубль': 6449, 'денькогда': 1672, 'какнакопить': 2422, 'получаться': 4232, 'решить': 5542, 'попробовать': 4362, 'одобрить': 3556, '12': 60, 'тысяча': 6473, 'беременный': 957, 'кровать': 2696, 'ребёнок': 5434, 'коляска': 2563, 'положение': 4223, 'финансовый': 6736, 'измениться': 2271, 'ужас': 6558, 'работыпомогать': 5182, 'мама': 2841, 'мочь': 2978, 'голод': 1538, 'умереть': 6603, 'возможность': 1267, 'платить': 4012, 'сам': 5736, 'виноватый': 1187, 'ещё': 1915, 'первый': 3892, 'позвонить': 4170, 'банк': 852, 'лежать': 2751, 'отделение': 3716, 'роддомародить': 5572, '20': 140, 'минута': 2922, 'назад': 3047, 'звонок': 2190, 'угроза': 6526, 'адрес': 676, 'мой': 2951, 'шокедовести': 6922, 'слеза': 5917, 'вроде': 1338, 'договориться': 1758, 'минимальный': 2916, 'платежахплатить': 4

In [ ]:
# теперь используем параметр n-gramm с обеими границами, равными 2
# по умолчанию параметр analyzer='word', поэтому указывать его не будем
vectorizer = CountVectorizer(ngram_range=(2,2))
sparse_matrix = vectorizer.fit_transform(data_bank_lemma['review'])
frequencies = sum(sparse_matrix).toarray()[0]
ngrams = pd.DataFrame(frequencies, index=vectorizer.get_feature_names_out(),
                                                          columns=['frequency'])
ngrams = ngrams.sort_values(by='frequency', ascending=False)
# выведем только первые 100 фраз, т.к. таблица выходит крайне большой (93407 стр)
ngrams[ngrams['frequency'] >= 100]

,frequency
тинькофф банк,450
кредитный карта,333
банк тинькофф,317
дебетовый карта,247
сотрудник банк,213
карта тинькофф,134
другой банк,131
клиент банк,129
личный кабинет,122
денежный средство,103


In [ ]:
# теперь используем параметр n-gramm с обеими границами, равными 2
vectorizer = CountVectorizer(ngram_range=(3,3))
sparse_matrix = vectorizer.fit_transform(data_bank_lemma['review'])
frequencies = sum(sparse_matrix).toarray()[0]
ngrams = pd.DataFrame(frequencies, index=vectorizer.get_feature_names_out(),
                                                          columns=['frequency'])
ngrams = ngrams.sort_values(by='frequency', ascending=False)
# снова выведем несколько первых фраз, т.к. таблица выходит еще больше (121659 стр)
ngrams[ngrams['frequency'] >= 10]

,frequency
являться клиент банк,50
кредитный карта тинькофф,42
пользоваться услуга банк,40
хотеть выразить благодарность,34
ао тинькофф банк,31
карта тинькофф банк,25
дебетовый карта тинькофф,24
открыть расчётный счёт,24
пользоваться дебетовый карта,24
сотрудник тинькофф банк,22


In [ ]:
# просто посмотреть, как работают границы ngram, т.к. на всех отзывах это будет не видно
# взяла 3 отзыв, потому что он выглядит интереснее
text = data_bank_lemma['review'].iloc[[2]]
model = CountVectorizer(ngram_range = (1, 4))
matrix = model.fit_transform(text).toarray()
df_output = pd.DataFrame(data = matrix, columns = model.get_feature_names_out())
df_output.T.head(10)

,0
банк,2
банк год,1
банк год иметь,1
банк год иметь дебетовый,1
банк захотеть,1
банк захотеть оптик,1
банк захотеть оптик камера,1
бац,1
бац отказ,1
бац отказ знать,1


#### TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

Далее векторизация будет проходить по датафрему с произведенной лемматизацией

In [ ]:
vectorizer = TfidfVectorizer()

vectorizer.fit(data_bank_lemma['review'])

print(vectorizer.vocabulary_)
print(vectorizer.idf_)
vector = vectorizer.transform(data_bank_lemma['review'])

print(vector.shape)
print(vector.toarray())

{'финансы': 14237, 'плохобыть': 9493, 'работа': 11261, 'муж': 7019, 'постоянный': 10251, 'доход': 4262, 'большой': 2237, 'работать': 11269, '36': 782, 'день': 3757, 'неделя': 7522, 'приносить': 10722, '6003000': 970, 'трубль': 13649, 'денькогда': 3782, 'какнакопить': 5550, 'получаться': 9966, 'решить': 11759, 'попробовать': 10138, 'одобрить': 8345, '12': 189, 'тысяча': 13703, 'беременный': 2069, 'кровать': 6260, 'ребёнок': 11651, 'коляска': 5937, 'положение': 9942, 'финансовый': 14236, 'измениться': 5199, 'ужас': 13869, 'работыпомогать': 11292, 'мама': 6628, 'мочь': 6997, 'голод': 3445, 'умереть': 13941, 'возможность': 2798, 'платить': 9454, 'сам': 11953, 'виноватый': 2637, 'ещё': 4424, 'первый': 9152, 'позвонить': 9824, 'банк': 1755, 'лежать': 6408, 'отделение': 8739, 'роддомародить': 11789, '20': 442, 'минута': 6843, 'назад': 7155, 'звонок': 5006, 'угроза': 13801, 'адрес': 1528, 'мой': 6934, 'шокедовести': 14668, 'слеза': 12349, 'вроде': 3001, 'договориться': 4042, 'минимальный': 682

In [ ]:
vectorizer = TfidfVectorizer()

vectorizer.fit(data_bank_lemma['review'])
vector = vectorizer.transform(data_bank_lemma['review'])
feature_names = vectorizer.get_feature_names_out()
tfidf_scores = vector.toarray()[0]
sorted_keywords = [word for _, word in sorted(zip(tfidf_scores, feature_names),
                                                                  reverse=True)]

print("Ключевые слова:", sorted_keywords)

Ключевые слова: ['тысяча', 'муж', 'мтс', 'ребёнок', 'спрашивать', 'получаться', 'оплатить', 'шокедовести', 'угрожаюторать', 'роддомародить', 'работыпомогать', 'понемногуно', 'плохобыть', 'платежахплатить', 'мошенница', 'кровать', 'коляска', 'какнакопить', 'звонокпредупредить', 'заведовать', 'дядинька', 'денькогда', 'беременный', 'банкеоплатить', '6003000', 'трубль', 'слеза', 'голод', '33', 'вернуть', 'выбивать', 'возможность', 'вообще', 'испортиться', 'умереть', 'уголовный', 'сейчас', 'ки', 'капать', '36', 'мозг', 'положение', '900', 'приносить', 'мама', 'финансы', 'быть', 'ужас', 'лежать', '34', 'ps', 'угроза', 'месяц', 'измениться', 'присылать', 'доход', 'виноватый', 'договориться', 'трубка', 'отделение', 'попробовать', '12', 'минимальный', 'деньга', 'постоянный', 'банк', 'вроде', 'финансовый', 'адрес', '20', 'далёкий', 'неделя', 'назад', 'полный', 'малосый', 'история', 'долг', 'одобрить', 'никто', 'приходить', 'брать', 'платить', 'ещё', 'большой', 'дело', 'хотя', 'сам', 'рубль', 'вс

#### word2vec

In [ ]:
import gensim
import gensim.downloader as download_api
from gensim.models.word2vec import Word2Vec

import numpy as np

In [ ]:
# w2v_model = Word2Vec(min_count=100, window=3)

In [ ]:
# w2v_model.build_vocab(data_bank_lemma['review'])

In [ ]:
# w2v_model.train(data_bank_lemma['review'], total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(6864906, 32947260)

In [ ]:
russian_model = download_api.load('word2vec-ruscorpora-300')

[==================================================] 100.0% 198.8/198.8MB downloaded


In [ ]:
print(russian_model.vector_size)
list(russian_model.index_to_key)[:20]

300


['весь_DET',
 'человек_NOUN',
 'мочь_VERB',
 'год_NOUN',
 'сказать_VERB',
 'время_NOUN',
 'говорить_VERB',
 'становиться_VERB',
 'знать_VERB',
 'самый_DET',
 'дело_NOUN',
 'день_NOUN',
 'жизнь_NOUN',
 'рука_NOUN',
 'очень_ADV',
 'первый_ADJ',
 'давать_VERB',
 'новый_ADJ',
 'слово_NOUN',
 'иметь_VERB']

In [ ]:
def get_review_vector(review):
    words = review.split()
    vector = np.zeros(russian_model.vector_size)
    count = 0
    for word in words:
        if word in russian_model:
            vector += russian_model[word]
            count += 1
    if count != 0:
        vector /= count
    return vector

In [ ]:
review_vectors = np.array([get_review_vector(review) for review in data_bank_lemma['review']])

In [ ]:
review_vectors

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Разделите отзывы на положительные и отрицательные можно на основании значения в поле «score» (например, 1,2- негативные, 4,5-позитивные).

In [ ]:
filtered_data = data_bank_lemma.copy()

filtered_data = data_bank_lemma[data_bank_lemma['score'].isin(['1', '2', '4', '5'])]
filtered_data['score'] = pd.to_numeric(filtered_data['score'])

positive_reviews = filtered_data[(filtered_data['score'].isin([4, 5]))]
negative_reviews = filtered_data[(filtered_data['score'].isin([1, 2]))]

filtered_data.head()

,bank_href,login,review,bank_name,site,date,message_href,score,status,user_href
6204,/services/responses/bank/tcs/,Vicky-13,счёт ип банк несколько день назад возникнуть п...,тинькофф банк,http://banki.ru,05.02.2019 18:13,NaN,5,Проверяется,/profile/?UID=230777
6205,/services/responses/bank/tcs/,loly,пользоваться банк достаточно давно кредитный д...,тинькофф банк,http://banki.ru,05.02.2019 18:09,NaN,5,Проверяется,/profile/?UID=125397
6206,/services/responses/bank/tcs/,9032489257,являться клиент банк ип физ лицо спасибо профе...,тинькофф банк,http://banki.ru,22.01.2019 13:58,NaN,5,Не засчитана,/profile/?UID=1579990
6208,/services/responses/bank/tcs/,b*******@mail.ru,общаться дважды специалист банк елизавета код ...,тинькофф банк,http://banki.ru,12.02.2019 12:21,NaN,5,Не засчитана,/profile/?UID=1687279
6210,/services/responses/bank/tcs/,маманаташа,пользоваться кредитный карта баланс небольшой ...,тинькофф банк,http://banki.ru,12.02.2019 12:27,NaN,5,Не засчитана,/profile/?UID=1687260


In [ ]:
print(f"Количество положительных отзывов: {positive_reviews.shape[0]}")
print(f"Количество отрицательных отзывов: {negative_reviews.shape[0]}")

Количество положительных отзывов: 765
Количество отрицательных отзывов: 331


### Классифицируйте отзывы на положительные/отрицательные с использованием разных векторных представлений. Можно использовать любую модель/модели класссификации.

In [ ]:
import warnings
warnings.simplefilter("ignore")

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# оставим только те строки, где есть данные по столбцу score
filtered_data = data_bank[data_bank['score'].isin(['1', '2', '4', '5'])]
filtered_data['score'] = pd.to_numeric(filtered_data['score'])

filtered_data.head()

,bank_href,login,review,bank_name,site,date,message_href,score,status,user_href
6204,/services/responses/bank/tcs/,Vicky-13,\n\t\tУ меня счет ИП в этом банке. несколько д...,тинькофф банк,http://banki.ru,05.02.2019 18:13,NaN,5,Проверяется,/profile/?UID=230777
6205,/services/responses/bank/tcs/,loly,\n\t\tПользуюсь банком достаточно давно. Как к...,тинькофф банк,http://banki.ru,05.02.2019 18:09,NaN,5,Проверяется,/profile/?UID=125397
6206,/services/responses/bank/tcs/,9032489257,\n\t\tЯвляюсь клиентом банка как ИП и физ. лиц...,тинькофф банк,http://banki.ru,22.01.2019 13:58,NaN,5,Не засчитана,/profile/?UID=1579990
6208,/services/responses/bank/tcs/,b*******@mail.ru,\n\t\tОбщалась дважды со специалистом банка Е...,тинькофф банк,http://banki.ru,12.02.2019 12:21,NaN,5,Не засчитана,/profile/?UID=1687279
6210,/services/responses/bank/tcs/,маманаташа,\n\t\tПользуюсь кредитной картой. Баланс на не...,тинькофф банк,http://banki.ru,12.02.2019 12:27,NaN,5,Не засчитана,/profile/?UID=1687260


Классификация для BoW с использованием наивного байесовского классификатора

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(filtered_data['review'])
y = filtered_data['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)

accuracy_nb_bow = accuracy_score(y_test, y_pred)

In [ ]:
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(filtered_data['review'])
y = filtered_data['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)

accuracy_lin_bow = accuracy_score(y_test, y_pred)

Классификация для TF-IDF с использованием наивного байесовского классификатора

In [ ]:
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(filtered_data['review'])
y = filtered_data['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)

accuracy_nb_idf = accuracy_score(y_test, y_pred)

In [ ]:
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(filtered_data['review'])
y = filtered_data['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)

accuracy_lin_idf = accuracy_score(y_test, y_pred)

Классификация для word2vec с использованием наивного байесовского классификатора

In [ ]:
review_vectors = np.array([get_review_vector(review) for review in filtered_data['review']])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(review_vectors,
                                                    filtered_data['score'],
                                                    test_size=0.2, random_state=42)

nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)

accuracy_nb_word = accuracy_score(y_test, y_pred)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(review_vectors,
                                                    filtered_data['score'],
                                                    test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)

accuracy_lin_word = accuracy_score(y_test, y_pred)

### Проанализируйте точность классификации при использовании разных векторных представлений.

In [ ]:
print(f"BoW ------->    NB: {round(accuracy_nb_bow, 3)};    LOGISTIC: {round(accuracy_lin_bow, 3)}" + "\n" +
      f"TF-IDF ---->    NB: {round(accuracy_nb_idf, 3)};    LOGISTIC: {round(accuracy_lin_idf, 3)}" + "\n" +
      f"word2vec -->    NB: {round(accuracy_nb_word, 3)};    LOGISTIC: {round(accuracy_lin_word, 3)}")

BoW ------->    NB: 0.827;    LOGISTIC: 0.827
TF-IDF ---->    NB: 0.645;    LOGISTIC: 0.814
word2vec -->    NB: 0.645;    LOGISTIC: 0.645


Таким образом, наибольшая точность видна при классификации отзывов, векторизация которых происходила с помощью подхода BoW

Показатели классификации для подхода TF-IDF повысились при использовании логистической регрессии.

В то же время подход word2vec показывал одинаковый результат для обоих случаев классификации.